In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools as itr
import doctest

In [2]:
N = 10
J = 1
s = np.ones((N,N), dtype=int)

In [3]:
beta = 0

i = np.random.randint(N)
j = np.random.randint(N)

s[i,j] *= -1

In [13]:
def energy(s):
    '''
    Returns the total energy of s.
    >>> energy(np.ones((10,10)))
    -400.0
    '''
    E = 0
    for i,j in itr.product(range(N),range(N)):
        E += s[i,j]*(s[(i+1)%N,j] + s[(i-1),j] + s[i,(j+1)%N] + s[i,(j-1)])
    return -J * E

In [14]:
energy(s)

-400

In [15]:
doctest.testmod()

TestResults(failed=0, attempted=1)